In [ ]:
%pip install -U langchain_community

In [1]:
import langchain_core, langchain_community
print(langchain_core.__version__)
print(langchain_community.__version__)

0.3.15
0.3.5


In [2]:
import ollama
from ollama import Message
from langchain_community.tools.tavily_search import TavilySearchResults

client = ollama.Client(host='http://localhost:11434')

In [3]:
# chat
client.chat("llama3.1", messages=[Message(role="user", content="Hey there!")], stream=False)

{'model': 'llama3.1',
 'created_at': '2024-11-02T04:41:39.1422069Z',
 'message': {'role': 'assistant',
  'content': "It's nice to meet you. Is there something I can help you with or would you like to chat?"},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 872554900,
 'load_duration': 19668900,
 'prompt_eval_count': 13,
 'prompt_eval_duration': 274509000,
 'eval_count': 23,
 'eval_duration': 577555000}

In [4]:
# streaming
for token in client.chat("llama3.1", messages=[Message(role="user", content="Hey there!")],stream=True):
    print(token['message']['content'], end="", flush=True)

It's nice to meet you. Is there something I can help you with or would you like to chat?

In [10]:
# tool calling
tool = TavilySearchResults(max_results=1)

def tavily_search(query:str)->str:
    return tool.invoke({"query":query})[0]['content']

tavily_search("What is the weather in New York City?")

"{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.714, 'lon': -74.006, 'tz_id': 'America/New_York', 'localtime_epoch': 1730522499, 'localtime': '2024-11-02 00:41'}, 'current': {'last_updated_epoch': 1730521800, 'last_updated': '2024-11-02 00:30', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 12.8, 'wind_kph': 20.5, 'wind_degree': 320, 'wind_dir': 'NW', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 51, 'cloud': 0, 'feelslike_c': 12.3, 'feelslike_f': 54.1, 'windchill_c': 11.6, 'windchill_f': 52.9, 'heatindex_c': 13.2, 'heatindex_f': 55.7, 'dewpoint_c': 4.2, 'dewpoint_f': 39.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 16.2, 'gust_kph': 26.1}}"

In [ ]:
available_tools = {"tavily_search": tavily_search}

response = client.chat(
    "llama3.1", 
    messages=[
        Message(role="user", content="What is the weather in New York City?")
    ],
    tools=[
        {
            "type": 'function',
            'function': {
                "name": 'tavily_search',
                "description": "Search up-to-date information on internet.",
                "parameters":{
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'string',
                            'description': 'a query to find on internet',
                        },
                    },
                    'required':['query'],
                },
            },
        },
    ],
    stream=False
)

for called_tool in response['message']['tool_calls']:
    function_to_call = available_tools[called_tool['function']['name']]
    function_response = function_to_call(**called_tool['function']['arguments'])
    messages = [
        Message(role="user", content="What is the weather in New York City?"),
        Message(role="tool", content=function_response)
    ]

final_response = client.chat(model="llama3.1", messages=messages)
print(final_response)

{'model': 'llama3.1', 'created_at': '2024-11-02T04:44:09.0753535Z', 'message': {'role': 'assistant', 'content': 'The current weather in New York City is:\n\n* Temperature: 57°F (13.9°C)\n* Conditions: Clear\n* Wind: 20.5 km/h (12.8 mph) from the NW at a pressure of 30.14 inches and humidity of 51%\n* Precipitation: None\n* Visibility: 16 kilometers (9 miles)\n\nNote: This is an automated response generated based on the provided data, please check other sources for more accurate and up-to-date information.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3695887500, 'load_duration': 13125900, 'prompt_eval_count': 414, 'prompt_eval_duration': 593064000, 'eval_count': 104, 'eval_duration': 3088555000}


In [13]:
print(final_response['message']['content'])

The current weather in New York City is:

* Temperature: 57°F (13.9°C)
* Conditions: Clear
* Wind: 20.5 km/h (12.8 mph) from the NW at a pressure of 30.14 inches and humidity of 51%
* Precipitation: None
* Visibility: 16 kilometers (9 miles)

Note: This is an automated response generated based on the provided data, please check other sources for more accurate and up-to-date information.
